In [27]:
from gensim import corpora
from gensim.models import LdaModel
import re,pickle,numpy

In [28]:
dictionary = corpora.Dictionary.load('LDA-dev.model.dict')
Lda = LdaModel.load('LDA-dev.model')
file = open('LDA-dev.model.stopwords.p', 'rb')
stopwords = pickle.load(file)
file.close()

In [29]:
dictionary

In [30]:
punctuation_string = ', . ! ? '
def getTopicForQuery (question):
    temp = question.lower()    
    for i in range(len(punctuation_string)):
        temp = temp.replace(punctuation_string[i], ' ')

    words = re.findall(r'\w+', temp, flags = re.UNICODE | re.LOCALE)
    
    important_words = []
    important_words = filter(lambda x: x not in stopwords, words)

    ques_vec = []
    ques_vec = dictionary.doc2bow(important_words)    
    
    topic_vec = []
    topic_vec = Lda[ques_vec]

    word_count_array = numpy.empty((len(topic_vec), 2), dtype = numpy.object)
    for i in range(len(topic_vec)):
        word_count_array[i, 0] = topic_vec[i][0]
        word_count_array[i, 1] = topic_vec[i][1]

    idx = numpy.argsort(word_count_array[:, 1])    
    idx = idx[::-1]
    word_count_array = word_count_array[idx]    
    
    final = Lda.print_topic(word_count_array[0, 0], 6)
    question_topic = ''
    for element in final.split(' + '):
        temp = element.split('*"')[1][:-1]
        question_topic+=" "+temp

    return topic_vec[0][0],question_topic

In [31]:
test = "swiss, franc, ollongren, snb, chamber, chf, witnesses, examine, answers, lelystad"
getTopicForQuery(test)

(92, ' swiss franc ollongren snb chamber chf')